In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random
pd.options.display.max_rows = 10
pd.options.display.max_columns = 500

In [2]:
DIVERSITY_UTILITY = 0.25
FRAC_ADMIT = 0.5


###
## Load dataframe where each row is a single applicant
###

df = pd.read_csv('./df_test.csv')
df['key'] = df['R'].astype(str) + "_"  + df['T'].astype(str)

df_stratum_utility = df[['R','T','B_p']].groupby(['R','T']).mean().reset_index()
df_stratum_utility['stratum_utility'] = (df_stratum_utility['B_p'] + DIVERSITY_UTILITY * df_stratum_utility['R']).round(2)
df_stratum_utility['key'] = df_stratum_utility['R'].astype(str) + "_" + df_stratum_utility['T'].astype(str)

df = df.merge(df_stratum_utility[['stratum_utility','key']], on='key')
df['ml_outcomes'] = df['stratum_utility']

In [3]:
FRAC_ADMIT = 0.5#df[['A']].sum()/len(df)


In [4]:
FRAC_ADMIT

0.5

In [5]:
len(df)

100000

In [6]:
#df['ml_outcomes'] = df['R']

In [7]:
##
# R = race, T = test score, ml_outcomes = expected utility from admitting, 
# ml_outcomes_{} counterfactual utility given race 
##

df[['R','T','ml_outcomes','T_black','T_white','B_p']].sort_values(by='ml_outcomes')



,R,T,ml_outcomes,T_black,T_white,B_p
98627,0,20,0.00,15,20,0
99515,0,19,0.00,17,19,0
99514,0,19,0.00,18,19,0
99920,0,15,0.00,13,15,0
99513,0,19,0.00,18,19,0
...,...,...,...,...,...,...
99369,1,92,1.25,92,105,1
99370,1,92,1.25,92,105,1
99371,1,92,1.25,92,108,1
99943,1,85,1.25,85,98,1


In [8]:
#df['ml_outcomes'] = df['ml_outcomes'] + 1*df['R']

In [9]:
###
## Get total utility from admitting people in a stratum 
###

df_ = df[['R','T','B_p','ml_outcomes']].groupby(['R','T','B_p']).sum().reset_index()

In [10]:
###
## Get number of people in each stratum
###

df_count = df[['R','T','ml_outcomes','B_p']].groupby(['R','T','B_p']).count().reset_index()
df_count.columns = ['R','T','B_p','Count']
df_count['N'] = df_count['Count']

In [11]:
###
## Merge summary tables to get one table with Race, Test Score, SUM(Utility), COUNT(applicants) per stratum
###

dff = df_.merge(df_count[['N']],left_index=True,right_index=True).sort_values(by='ml_outcomes',ascending=False).reset_index().sort_values(by='index').reset_index()

In [12]:
# final info table
dff.sort_values(by='ml_outcomes')

,level_0,index,R,T,B_p,ml_outcomes,N
0,332,0,0,14,0,0.00,1
1,327,1,0,15,0,0.00,2
2,331,2,0,16,0,0.00,2
5,330,5,0,18,0,0.00,6
6,329,6,0,19,0,0.00,10
...,...,...,...,...,...,...,...
95,4,95,0,64,1,642.53,959
101,3,101,0,67,1,661.38,906
99,2,99,0,66,1,666.69,939
93,1,93,0,63,1,693.66,1051


### Setup optimization problem 

In [13]:
from ortools.linear_solver import pywraplp


In [14]:
solver = pywraplp.Solver.CreateSolver('GLOP')


In [15]:
dff

,level_0,index,R,T,B_p,ml_outcomes,N
0,332,0,0,14,0,0.00,1
1,327,1,0,15,0,0.00,2
2,331,2,0,16,0,0.00,2
3,305,3,0,17,0,1.20,3
4,317,4,0,17,1,0.80,2
...,...,...,...,...,...,...,...
328,281,328,1,95,1,2.50,2
329,280,329,1,97,1,2.50,2
330,302,330,1,98,1,1.25,1
331,303,331,1,99,1,1.25,1


In [16]:
applicant_stratum = []
vars_cache = {}

# Objective: Maximize the expected utility of the admitted students
objective = solver.Objective()

# For each stratum
for ix, row in dff.iterrows():
    # probability of admission
    numvar = solver.NumVar(0.0, 1.0, str(ix))
    
    # store variable by index, and also by stratum R, T
    applicant_stratum.append(numvar)
    vars_cache[(row['R'],row['T'],row['B_p'])] = numvar
    
    # Benefit of admitting people is total utility in that stratum
    objective.SetCoefficient(applicant_stratum[ix], float(row['ml_outcomes']))
objective.SetMaximization()


In [17]:
# Currently we have no constraints 
solver.NumConstraints()

0

In [18]:
# Constraint: At most K applicants
K = int(len(df)*FRAC_ADMIT)
print(K)
admit_quota = solver.Constraint(0, K)

# Total applicants cannot exceed K 
for ix, row in dff.iterrows():
    admit_quota.SetCoefficient(applicant_stratum[ix], float(row['N']))

50000


In [19]:
# Now we have one constraint
solver.NumConstraints()

1

## Add Equalized Odds Constraints

In [20]:
## Make sure that you have to add all people in B_p stratum or none
## i.e. you can't add only people who pass boards and reject those who fail boards from same T, R stratum
didntexist, exists = 0, 0 

for ix, row in dff.iterrows():
    constrain_bp = solver.Constraint(0.0, 0.0)
    
    var1 = vars_cache[(row['R'],row['T'],row['B_p'])]
    key2 = (row['R'],row['T'], 1-row['B_p'])
    
    if key2 not in vars_cache:
        didntexist+=1
        continue
        
    var2 = vars_cache[key2]
    
    constrain_bp.SetCoefficient(var1, -1.0)
    constrain_bp.SetCoefficient(var2, 1.0)
    exists+=1

didntexist, exists

(51, 282)

In [21]:
white_pass_boards = []
white_fail_boards = []
black_pass_boards = []
black_fail_boards = []

for key in vars_cache:
    r, t, b_p = key
    if b_p == 1 and r==0:
        white_pass_boards.append(key)
    elif b_p == 0 and r==0:
        white_fail_boards.append(key)
    elif b_p == 1 and r==1:
        black_pass_boards.append(key)
    elif b_p == 0 and r==1:
        black_fail_boards.append(key)

len(white_pass_boards),len(white_fail_boards),len(black_pass_boards),len(black_fail_boards)

(99, 83, 79, 72)

In [22]:
NUM_TOTALS = {}
df_totals = dff[['N','R','B_p']].groupby(['R','B_p']).sum().reset_index()
for ix, row in df_totals.iterrows():
    NUM_TOTALS[(row['R'],row['B_p'])] = row['N']
    
N_IN_STRATAS = {}
for ix, row in dff.iterrows():
    N_IN_STRATAS[(row['R'],row['T'],row['B_p'])] = row['N']

In [23]:
# Now we have one constraint
solver.NumConstraints()

334

In [24]:
#Of those who pass the boards exams
#Frac white admitted and frac black admitted should be the same

constrain_pass_boards = solver.Constraint(0.0, 0.0)

for key in white_pass_boards:
    r, t, b_p = key
    N_IN_STRATUM = N_IN_STRATAS[(r,t,b_p)]
    N_TOTAL = NUM_TOTALS[(r,b_p)]
    
    constrain_pass_boards.SetCoefficient(vars_cache[key], float(N_IN_STRATUM) / float(N_TOTAL))

for key in black_pass_boards:
    r, t, b_p = key
    N_IN_STRATUM = N_IN_STRATAS[(r,t,b_p)]
    N_TOTAL = NUM_TOTALS[(r,b_p)]
    
    constrain_pass_boards.SetCoefficient(vars_cache[key], -1.0 * (float(N_IN_STRATUM) / float(N_TOTAL)))


In [25]:
#Of those who fail the boards exams
#Frac white admitted and frac black admitted should be the same

constrain_fail_boards = solver.Constraint(0.0, 0.0)

for key in white_fail_boards:
    r, t, b_p = key
    N_IN_STRATUM = N_IN_STRATAS[(r,t,b_p)]
    N_TOTAL = NUM_TOTALS[(r,b_p)]
    
    constrain_fail_boards.SetCoefficient(vars_cache[key], float(N_IN_STRATUM) / float(N_TOTAL))

for key in black_fail_boards:
    r, t, b_p = key
    N_IN_STRATUM = N_IN_STRATAS[(r,t,b_p)]
    N_TOTAL = NUM_TOTALS[(r,b_p)]
    
    constrain_fail_boards.SetCoefficient(vars_cache[key], -1.0 * (float(N_IN_STRATUM) / float(N_TOTAL)))


## Solve linear program

In [26]:
solver.ABNORMAL

4

In [27]:
status = solver.Solve()


In [28]:
status

0

In [29]:
solver.OPTIMAL

0

In [30]:
row = []
admit = []

for i in applicant_stratum:
    row.append(int(str(i)))
    admit.append(i.solution_value())

df_decisions = pd.DataFrame({'row_id':row,'decision':admit})

In [31]:
df_decisions

,row_id,decision
0,0,0.0
1,1,0.0
2,2,0.0
3,3,0.0
4,4,0.0
...,...,...
328,328,1.0
329,329,1.0
330,330,1.0
331,331,1.0


In [32]:
dff.merge(df_decisions,left_index=True,right_index=True).sort_values(by='ml_outcomes',ascending=False)

,level_0,index,R,T,B_p,ml_outcomes,N,row_id,decision
89,0,89,0,61,1,709.76,1109,89,1.0
93,1,93,0,63,1,693.66,1051,93,1.0
99,2,99,0,66,1,666.69,939,99,1.0
101,3,101,0,67,1,661.38,906,101,1.0
95,4,95,0,64,1,642.53,959,95,1.0
...,...,...,...,...,...,...,...,...,...
7,328,7,0,20,0,0.00,11,7,0.0
6,329,6,0,19,0,0.00,10,6,0.0
5,330,5,0,18,0,0.00,6,5,0.0
2,331,2,0,16,0,0.00,2,2,0.0


In [33]:
xxx_ = dff.merge(df_decisions,left_index=True,right_index=True).sort_values(by='ml_outcomes',ascending=False)
xxx_.sort_values(by='T')

,level_0,index,R,T,B_p,ml_outcomes,N,row_id,decision
182,322,182,1,11,0,0.25,1,182,0.0
183,319,183,1,12,0,0.50,2,183,0.0
184,323,184,1,14,0,0.25,1,184,0.0
0,332,0,0,14,0,0.00,1,0,0.0
1,327,1,0,15,0,0.00,2,1,0.0
...,...,...,...,...,...,...,...,...,...
177,287,177,0,114,1,2.00,2,177,1.0
178,288,178,0,115,1,2.00,2,178,1.0
179,314,179,0,116,1,1.00,1,179,1.0
180,313,180,0,120,1,1.00,1,180,1.0


In [34]:
xxx = dff.merge(df_decisions,left_index=True,right_index=True).sort_values(by='ml_outcomes',ascending=False)
xxx

,level_0,index,R,T,B_p,ml_outcomes,N,row_id,decision
89,0,89,0,61,1,709.76,1109,89,1.0
93,1,93,0,63,1,693.66,1051,93,1.0
99,2,99,0,66,1,666.69,939,99,1.0
101,3,101,0,67,1,661.38,906,101,1.0
95,4,95,0,64,1,642.53,959,95,1.0
...,...,...,...,...,...,...,...,...,...
7,328,7,0,20,0,0.00,11,7,0.0
6,329,6,0,19,0,0.00,10,6,0.0
5,330,5,0,18,0,0.00,6,5,0.0
2,331,2,0,16,0,0.00,2,2,0.0


In [35]:
xxx.to_csv('./decision.csv')

In [36]:
solution = pd.read_csv('./decision.csv')

In [37]:
xxx

,level_0,index,R,T,B_p,ml_outcomes,N,row_id,decision
89,0,89,0,61,1,709.76,1109,89,1.0
93,1,93,0,63,1,693.66,1051,93,1.0
99,2,99,0,66,1,666.69,939,99,1.0
101,3,101,0,67,1,661.38,906,101,1.0
95,4,95,0,64,1,642.53,959,95,1.0
...,...,...,...,...,...,...,...,...,...
7,328,7,0,20,0,0.00,11,7,0.0
6,329,6,0,19,0,0.00,10,6,0.0
5,330,5,0,18,0,0.00,6,5,0.0
2,331,2,0,16,0,0.00,2,2,0.0


In [38]:
df['key'] = df['R'].astype(str)+'_'+df['T'].astype(str)+'_'+df['B_p'].astype(str)
xxx['key'] = xxx['R'].astype(str)+'_'+xxx['T'].astype(str)+'_'+xxx['B_p'].astype(str)

In [39]:
xxx

,level_0,index,R,T,B_p,ml_outcomes,N,row_id,decision,key
89,0,89,0,61,1,709.76,1109,89,1.0,0_61_1
93,1,93,0,63,1,693.66,1051,93,1.0,0_63_1
99,2,99,0,66,1,666.69,939,99,1.0,0_66_1
101,3,101,0,67,1,661.38,906,101,1.0,0_67_1
95,4,95,0,64,1,642.53,959,95,1.0,0_64_1
...,...,...,...,...,...,...,...,...,...,...
7,328,7,0,20,0,0.00,11,7,0.0,0_20_0
6,329,6,0,19,0,0.00,10,6,0.0,0_19_0
5,330,5,0,18,0,0.00,6,5,0.0,0_18_0
2,331,2,0,16,0,0.00,2,2,0.0,0_16_0


In [40]:
admit_decisions = df.merge(xxx[['key','decision']],how='left',on='key')
admit_decisions['decision_random'] = pd.Series([random.random() for x in range(0,len(admit_decisions))]) < FRAC_ADMIT

FRAC_BLACK_POLICY = (admit_decisions['R'] * admit_decisions['decision']).sum()/admit_decisions['decision'].sum()
SUM_BP_POLICY = ( admit_decisions['B_p'] * admit_decisions['decision']).sum()

FRAC_RANDOM_POLICY = (admit_decisions['R'] * admit_decisions['decision_random']).sum()/admit_decisions['decision_random'].sum()
SUM_BP_RAND_POLICY = ( admit_decisions['B_p'] * admit_decisions['decision_random']).sum()

In [41]:
file = open('./lp_results.csv','a')
file.write('{}\t{}\t{}\n'.format('Counterfactual Equalized Odds',str(FRAC_BLACK_POLICY),str(SUM_BP_POLICY)))
file.close()

In [42]:
xxx[xxx['R']==0].sort_values(by='decision')

,level_0,index,R,T,B_p,ml_outcomes,N,row_id,decision,key
0,332,0,0,14,0,0.00,1,0,0.0,0_14_0
61,95,61,0,47,1,222.02,653,61,0.0,0_47_1
48,89,48,0,41,0,239.46,921,48,0.0,0_41_0
32,172,32,0,33,0,61.88,364,32,0.0,0_33_0
41,207,41,0,37,1,26.10,145,41,0.0,0_37_1
...,...,...,...,...,...,...,...,...,...,...
108,131,108,0,71,0,134.30,170,108,1.0,0_71_0
110,135,110,0,72,0,119.07,147,110,1.0,0_72_0
141,137,141,0,87,1,117.12,122,141,1.0,0_87_1
104,109,104,0,69,0,188.25,251,104,1.0,0_69_0


In [43]:
xxx[xxx['R']==1].sort_values(by='decision')

,level_0,index,R,T,B_p,ml_outcomes,N,row_id,decision,key
240,27,240,1,45,0,497.12,956,240,0.0,1_45_0
198,206,198,1,24,0,26.35,85,198,0.0,1_24_0
219,195,219,1,34,1,32.37,83,219,0.0,1_34_1
221,188,221,1,35,1,39.00,100,221,0.0,1_35_1
223,187,223,1,36,1,39.52,104,223,0.0,1_36_1
...,...,...,...,...,...,...,...,...,...,...
266,87,266,1,58,0,240.92,317,266,1.0,1_58_0
253,90,253,1,51,1,239.40,380,253,1.0,1_51_1
282,170,282,1,66,0,66.74,71,282,1.0,1_66_0
284,166,284,1,67,0,69.12,72,284,1.0,1_67_0


In [44]:
(xxx['N']*xxx['decision']).sum()/len(df)

0.5000000000000001

In [45]:
admit_decisions[admit_decisions['B_p']==1][['decision','R']].groupby(['R']).mean()

,decision
R,
0,0.682909
1,0.682909
